# PyCity Schools Analysis

This code analyzes various data from two csv files about a school district by breaking it down into different pivot tables and by calculating various averages and sums. The code analyzes test score data for different schools within the district and profiles them by size, school type and budget/spending.

One thing I noticed while writing this code was that Charter schools had much higher passing rates in reading, math and combined test scores. I also noticed that schools with higher spending per student did not necessarily equate to higher percentages of students passing both subjects overall. 
  
---

In [459]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

school_data_complete.columns

In [460]:
# Calculate the total number of unique schools
school_count = len(school_data['school_name'])

school_count


15

In [461]:
# Calculate the total number of students
student_count = len(student_data['student_name'])
student_count

39170

In [462]:
# Calculate the total budget
total_budget = school_data['budget']
total_budget.sum()



24649428

In [463]:
# Calculate the average (mean) math score
average_math_score = student_data['math_score']
average_math_score.mean()

78.98537145774827

In [464]:
# Calculate the average (mean) reading score
average_reading_score = student_data['reading_score']
average_reading_score.mean()

81.87784018381414

In [465]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [466]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete['reading_score']>= 70)].count()['student_name']
passing_reading_percentage = passing_reading_count/float(student_count)*100
print (passing_reading_percentage,"% of students passed reading")

85.80546336482001 % of students passed reading


In [467]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [468]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Number of Schools":[school_count],
    "Total Students":[student_count],
    "Total Budget":[total_budget.sum()],
    "Average Math Score":[average_math_score.mean()],
    "Average Reading Score":[average_reading_score.mean()],
    "Math Passing Rate":[passing_math_percentage],
    "Reading Passing Rate":[passing_reading_percentage],
    "Overall Passing Rate":[overall_passing_rate]})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:,.2f}%".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:,.2f}%".format)
district_summary["Math Passing Rate"] = district_summary["Math Passing Rate"].map("{:,.2f}%".format)
district_summary["Reading Passing Rate"] = district_summary["Reading Passing Rate"].map("{:,.2f}%".format)
district_summary["Overall Passing Rate"] = district_summary["Overall Passing Rate"].map("{:,.2f}%".format)
# Display the DataFrame
district_summary

,Number of Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99%,81.88%,74.98%,85.81%,65.17%


## School Summary

In [469]:
# Use the code provided to select all of the school types
school_types = (school_data['type'])
school_types

0     District
1     District
2      Charter
3     District
4      Charter
5      Charter
6      Charter
7     District
8      Charter
9      Charter
10     Charter
11    District
12    District
13    District
14     Charter
Name: type, dtype: object

In [470]:
# Calculate the total student count per school
per_school_counts = school_data_complete.groupby(['school_name'])

per_school_counts[['student_name']].count()

,student_name
school_name,
Bailey High School,4976
Cabrera High School,1858
Figueroa High School,2949
Ford High School,2739
Griffin High School,1468
Hernandez High School,4635
Holden High School,427
Huang High School,2917
Johnson High School,4761


In [471]:
# Calculate the total school budget and per capita spending per school

per_school_budget = per_school_counts['budget']
per_school_capita = ((school_data_complete['budget'])/(school_data_complete['size']))

school_data_complete['Per Student Spending']=per_school_capita

new_df = per_school_counts[['budget','type','Per Student Spending']].value_counts().reset_index()

new_df = new_df.rename(columns={"count":"Number of Students","school_name":"School Name",'budget':'Budget','type':'School Type'})

new_df




,School Name,Budget,School Type,Per Student Spending,Number of Students
0,Bailey High School,3124928,District,628.0,4976
1,Cabrera High School,1081356,Charter,582.0,1858
2,Figueroa High School,1884411,District,639.0,2949
3,Ford High School,1763916,District,644.0,2739
4,Griffin High School,917500,Charter,625.0,1468
5,Hernandez High School,3022020,District,652.0,4635
6,Holden High School,248087,Charter,581.0,427
7,Huang High School,1910635,District,655.0,2917
8,Johnson High School,3094650,District,650.0,4761
9,Pena High School,585858,Charter,609.0,962


In [472]:
# Calculate the average test scores per school
per_school_math = per_school_counts['math_score'].mean()
per_school_reading = per_school_counts['reading_score'].mean()

new_df['Average Math Score'] = per_school_math.values
new_df['Average Reading Score'] = per_school_reading.values

new_df

,School Name,Budget,School Type,Per Student Spending,Number of Students,Average Math Score,Average Reading Score
0,Bailey High School,3124928,District,628.0,4976,77.048432,81.033963
1,Cabrera High School,1081356,Charter,582.0,1858,83.061895,83.975780
2,Figueroa High School,1884411,District,639.0,2949,76.711767,81.158020
3,Ford High School,1763916,District,644.0,2739,77.102592,80.746258
4,Griffin High School,917500,Charter,625.0,1468,83.351499,83.816757
5,Hernandez High School,3022020,District,652.0,4635,77.289752,80.934412
6,Holden High School,248087,Charter,581.0,427,83.803279,83.814988
7,Huang High School,1910635,District,655.0,2917,76.629414,81.182722
8,Johnson High School,3094650,District,650.0,4761,77.072464,80.966394
9,Pena High School,585858,Charter,609.0,962,83.839917,84.044699


In [473]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_data_complete['math_score'] >= 70


school_data_complete['Students Passing Math'] = students_passing_math

# Group the data by school and count the number of students passing math in each school
school_students_passing_math = per_school_counts[['Students Passing Math']].sum()

# Display the number of students passing math per school
new_df["Students Passing Math"]=school_students_passing_math.values

new_df


,School Name,Budget,School Type,Per Student Spending,Number of Students,Average Math Score,Average Reading Score,Students Passing Math
0,Bailey High School,3124928,District,628.0,4976,77.048432,81.033963,3318
1,Cabrera High School,1081356,Charter,582.0,1858,83.061895,83.975780,1749
2,Figueroa High School,1884411,District,639.0,2949,76.711767,81.158020,1946
3,Ford High School,1763916,District,644.0,2739,77.102592,80.746258,1871
4,Griffin High School,917500,Charter,625.0,1468,83.351499,83.816757,1371
5,Hernandez High School,3022020,District,652.0,4635,77.289752,80.934412,3094
6,Holden High School,248087,Charter,581.0,427,83.803279,83.814988,395
7,Huang High School,1910635,District,655.0,2917,76.629414,81.182722,1916
8,Johnson High School,3094650,District,650.0,4761,77.072464,80.966394,3145
9,Pena High School,585858,Charter,609.0,962,83.839917,84.044699,910


In [474]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_data_complete['reading_score']>=70

school_data_complete['Students Passing Reading'] = students_passing_reading

# Group the data by school and count the number of students passing math in each school
school_students_passing_reading =  per_school_counts[['Students Passing Reading']].sum()

new_df["Students Passing Reading"]=school_students_passing_reading.values

new_df

,School Name,Budget,School Type,Per Student Spending,Number of Students,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading
0,Bailey High School,3124928,District,628.0,4976,77.048432,81.033963,3318,4077
1,Cabrera High School,1081356,Charter,582.0,1858,83.061895,83.975780,1749,1803
2,Figueroa High School,1884411,District,639.0,2949,76.711767,81.158020,1946,2381
3,Ford High School,1763916,District,644.0,2739,77.102592,80.746258,1871,2172
4,Griffin High School,917500,Charter,625.0,1468,83.351499,83.816757,1371,1426
5,Hernandez High School,3022020,District,652.0,4635,77.289752,80.934412,3094,3748
6,Holden High School,248087,Charter,581.0,427,83.803279,83.814988,395,411
7,Huang High School,1910635,District,655.0,2917,76.629414,81.182722,1916,2372
8,Johnson High School,3094650,District,650.0,4761,77.072464,80.966394,3145,3867
9,Pena High School,585858,Charter,609.0,962,83.839917,84.044699,910,923


In [475]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

new_df['Students Passing Both'] = school_students_passing_math_and_reading.values

new_df

,School Name,Budget,School Type,Per Student Spending,Number of Students,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Students Passing Both
0,Bailey High School,3124928,District,628.0,4976,77.048432,81.033963,3318,4077,2719
1,Cabrera High School,1081356,Charter,582.0,1858,83.061895,83.975780,1749,1803,1697
2,Figueroa High School,1884411,District,639.0,2949,76.711767,81.158020,1946,2381,1569
3,Ford High School,1763916,District,644.0,2739,77.102592,80.746258,1871,2172,1487
4,Griffin High School,917500,Charter,625.0,1468,83.351499,83.816757,1371,1426,1330
5,Hernandez High School,3022020,District,652.0,4635,77.289752,80.934412,3094,3748,2481
6,Holden High School,248087,Charter,581.0,427,83.803279,83.814988,395,411,381
7,Huang High School,1910635,District,655.0,2917,76.629414,81.182722,1916,2372,1561
8,Johnson High School,3094650,District,650.0,4761,77.072464,80.966394,3145,3867,2549
9,Pena High School,585858,Charter,609.0,962,83.839917,84.044699,910,923,871


In [476]:
# Use the provided code to calculate the passing rates
per_school_passing_math = new_df['Students Passing Math'] / new_df['Number of Students'] * 100
per_school_passing_reading = new_df['Students Passing Reading'] / new_df['Number of Students'] * 100
overall_passing_rate = new_df['Students Passing Both'] / new_df['Number of Students'] * 100

new_df['Students Passing Math Percent']=per_school_passing_math.values
new_df['Students Passing Reading Percent']=per_school_passing_reading.values
new_df['Students Passing Both Percent']=overall_passing_rate.values

new_df.head()



,School Name,Budget,School Type,Per Student Spending,Number of Students,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Students Passing Both,Students Passing Math Percent,Students Passing Reading Percent,Students Passing Both Percent
0,Bailey High School,3124928,District,628.0,4976,77.048432,81.033963,3318,4077,2719,66.680064,81.933280,54.642283
1,Cabrera High School,1081356,Charter,582.0,1858,83.061895,83.975780,1749,1803,1697,94.133477,97.039828,91.334769
2,Figueroa High School,1884411,District,639.0,2949,76.711767,81.158020,1946,2381,1569,65.988471,80.739234,53.204476
3,Ford High School,1763916,District,644.0,2739,77.102592,80.746258,1871,2172,1487,68.309602,79.299014,54.289887
4,Griffin High School,917500,Charter,625.0,1468,83.351499,83.816757,1371,1426,1330,93.392371,97.138965,90.599455


In [477]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = new_df[['School Name','School Type','Budget','Number of Students',"Per Student Spending",
       'Average Math Score', 'Average Reading Score',
       'Students Passing Math Percent', 'Students Passing Reading Percent',
       'Students Passing Both Percent']]

# Formatting
per_school_summary["Budget"] = per_school_summary["Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Spending"] = per_school_summary["Per Student Spending"].map("${:,.2f}".format)
per_school_summary["Students Passing Math Percent"] = per_school_summary["Students Passing Math Percent"].map("{:,.2f}%".format)
per_school_summary["Students Passing Reading Percent"] = per_school_summary["Students Passing Reading Percent"].map("{:,.2f}%".format)
per_school_summary["Students Passing Both Percent"] = per_school_summary["Students Passing Both Percent"].map("{:,.2f}%".format)

# Display the DataFrame
per_school_summary

C:\Users\deebn\AppData\Local\Temp\ipykernel_17528\1276004308.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_school_summary["Budget"] = per_school_summary["Budget"].map("${:,.2f}".format)
C:\Users\deebn\AppData\Local\Temp\ipykernel_17528\1276004308.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_school_summary["Per Student Spending"] = per_school_summary["Per Student Spending"].map("${:,.2f}".format)
C:\Users\deebn\AppData\Local\Temp\ipykernel_17528\1276004308.py:10: SettingWithCopyWarning: 

,School Name,School Type,Budget,Number of Students,Per Student Spending,Average Math Score,Average Reading Score,Students Passing Math Percent,Students Passing Reading Percent,Students Passing Both Percent
0,Bailey High School,District,"$3,124,928.00",4976,$628.00,77.048432,81.033963,66.68%,81.93%,54.64%
1,Cabrera High School,Charter,"$1,081,356.00",1858,$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
2,Figueroa High School,District,"$1,884,411.00",2949,$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
3,Ford High School,District,"$1,763,916.00",2739,$644.00,77.102592,80.746258,68.31%,79.30%,54.29%
4,Griffin High School,Charter,"$917,500.00",1468,$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
5,Hernandez High School,District,"$3,022,020.00",4635,$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
6,Holden High School,Charter,"$248,087.00",427,$581.00,83.803279,83.814988,92.51%,96.25%,89.23%
7,Huang High School,District,"$1,910,635.00",2917,$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
8,Johnson High School,District,"$3,094,650.00",4761,$650.00,77.072464,80.966394,66.06%,81.22%,53.54%
9,Pena High School,Charter,"$585,858.00",962,$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


## Highest-Performing Schools (by % Overall Passing)

In [478]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by='Students Passing Both Percent', ascending=False)
top_schools.head(5)

,School Name,School Type,Budget,Number of Students,Per Student Spending,Average Math Score,Average Reading Score,Students Passing Math Percent,Students Passing Reading Percent,Students Passing Both Percent
1,Cabrera High School,Charter,"$1,081,356.00",1858,$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
12,Thomas High School,Charter,"$1,043,130.00",1635,$638.00,83.418349,83.848930,93.27%,97.31%,90.95%
4,Griffin High School,Charter,"$917,500.00",1468,$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
13,Wilson High School,Charter,"$1,319,574.00",2283,$578.00,83.274201,83.989488,93.87%,96.54%,90.58%
9,Pena High School,Charter,"$585,858.00",962,$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

In [479]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by='Students Passing Both Percent', ascending=True)
bottom_schools.head(5)

,School Name,School Type,Budget,Number of Students,Per Student Spending,Average Math Score,Average Reading Score,Students Passing Math Percent,Students Passing Reading Percent,Students Passing Both Percent
10,Rodriguez High School,District,"$2,547,363.00",3999,$637.00,76.842711,80.744686,66.37%,80.22%,52.99%
2,Figueroa High School,District,"$1,884,411.00",2949,$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
7,Huang High School,District,"$1,910,635.00",2917,$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
5,Hernandez High School,District,"$3,022,020.00",4635,$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
8,Johnson High School,District,"$3,094,650.00",4761,$650.00,77.072464,80.966394,66.06%,81.22%,53.54%


## Math Scores by Grade

In [480]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby("school_name")["math_score"].mean()
tenth_grader_math_scores = tenth_graders.groupby("school_name")["math_score"].mean()
eleventh_grader_math_scores = eleventh_graders.groupby("school_name")["math_score"].mean()
twelfth_grader_math_scores = twelfth_graders.groupby("school_name")["math_score"].mean()

# Combine each of the scores above into a single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({
    "9th Grade Math Scores": ninth_grade_math_scores,
    "10th Grade Math Scores": tenth_grader_math_scores,
    "11th Grade Math Scores": eleventh_grader_math_scores,
    "12th Grade Math Scores": twelfth_grader_math_scores
})

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade


,9th Grade Math Scores,10th Grade Math Scores,11th Grade Math Scores,12th Grade Math Scores
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [481]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[school_data_complete["grade"] == "9th"]
tenth_graders = school_data_complete[school_data_complete["grade"] == "10th"]
eleventh_graders = school_data_complete[school_data_complete["grade"] == "11th"]
twelfth_graders = school_data_complete[school_data_complete["grade"] == "12th"]

# Group by `school_name` and take the mean of the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby("school_name")["reading_score"].mean()
tenth_grade_reading_scores = tenth_graders.groupby("school_name")["reading_score"].mean()
eleventh_grade_reading_scores = eleventh_graders.groupby("school_name")["reading_score"].mean()
twelfth_grade_reading_scores = twelfth_graders.groupby("school_name")["reading_score"].mean()

# Combine each of the scores above into a single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({
    "9th Grade Reading Scores": ninth_grade_reading_scores,
    "10th Grade Reading Scores": tenth_grade_reading_scores,
    "11th Grade Reading Scores": eleventh_grade_reading_scores,
    "12th Grade Reading Scores": twelfth_grade_reading_scores
})

# Minor data wrangling
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade


,9th Grade Reading Scores,10th Grade Reading Scores,11th Grade Reading Scores,12th Grade Reading Scores
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [497]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [498]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [506]:

# Use `pd.cut` to categorize spending based on the bins
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Spending"], bins=spending_bins, labels=labels)

# Display the DataFrame with the new column
school_spending_df


,School Name,School Type,Budget,Number of Students,Per Student Spending,Average Math Score,Average Reading Score,Students Passing Math Percent,Students Passing Reading Percent,Students Passing Both Percent,Spending Ranges (Per Student)
0,Bailey High School,District,"$3,124,928.00",4976,628.0,77.048432,81.033963,66.68,81.93,54.64,$585-630
1,Cabrera High School,Charter,"$1,081,356.00",1858,582.0,83.061895,83.975780,94.13,97.04,91.33,<$585
2,Figueroa High School,District,"$1,884,411.00",2949,639.0,76.711767,81.158020,65.99,80.74,53.20,$630-645
3,Ford High School,District,"$1,763,916.00",2739,644.0,77.102592,80.746258,68.31,79.30,54.29,$630-645
4,Griffin High School,Charter,"$917,500.00",1468,625.0,83.351499,83.816757,93.39,97.14,90.60,$585-630
5,Hernandez High School,District,"$3,022,020.00",4635,652.0,77.289752,80.934412,66.75,80.86,53.53,$645-680
6,Holden High School,Charter,"$248,087.00",427,581.0,83.803279,83.814988,92.51,96.25,89.23,<$585
7,Huang High School,District,"$1,910,635.00",2917,655.0,76.629414,81.182722,65.68,81.32,53.51,$645-680
8,Johnson High School,District,"$3,094,650.00",4761,650.0,77.072464,80.966394,66.06,81.22,53.54,$645-680
9,Pena High School,Charter,"$585,858.00",962,609.0,83.839917,84.044699,94.59,95.95,90.54,$585-630


In [512]:
# Convert the percentage columns to strings and remove '%' symbols
school_spending_df["Students Passing Math Percent"] = school_spending_df["Students Passing Math Percent"].astype(str).str.rstrip('%').astype(float)
school_spending_df["Students Passing Reading Percent"] = school_spending_df["Students Passing Reading Percent"].astype(str).str.rstrip('%').astype(float)
school_spending_df["Students Passing Both Percent"] = school_spending_df["Students Passing Both Percent"].astype(str).str.rstrip('%').astype(float)

# Calculate averages for the desired columns
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Students Passing Math Percent"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Students Passing Reading Percent"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Students Passing Both Percent"].mean()



In [513]:
# Create a DataFrame to store the averages
spending_summary_df = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "Average % Passing Math": spending_passing_math,
    "Average % Passing Reading": spending_passing_reading,
    "Average % Overall Passing": overall_passing_spending
})

# Display the DataFrame
spending_summary_df

,Average Math Score,Average Reading Score,Average % Passing Math,Average % Passing Reading,Average % Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460000,96.610000,90.367500
$585-630,81.899826,83.155286,87.132500,92.717500,81.417500
$630-645,78.518855,81.624473,73.485000,84.392500,62.857500
$645-680,76.997210,81.027843,66.163333,81.133333,53.526667


## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [525]:
import pandas as pd

# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Use `pd.cut` to categorize schools based on size
per_school_summary["School Size"] = pd.cut(per_school_summary["Number of Students"], bins=size_bins, labels=labels)

# Display the updated DataFrame
per_school_summary


C:\Users\deebn\AppData\Local\Temp\ipykernel_17528\3133547511.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_school_summary["School Size"] = pd.cut(per_school_summary["Number of Students"], bins=size_bins, labels=labels)


,School Name,School Type,Budget,Number of Students,Per Student Spending,Average Math Score,Average Reading Score,Students Passing Math Percent,Students Passing Reading Percent,Students Passing Both Percent,School Size
0,Bailey High School,District,"$3,124,928.00",4976,$628.00,77.048432,81.033963,66.68,81.93,54.64,Large (2000-5000)
1,Cabrera High School,Charter,"$1,081,356.00",1858,$582.00,83.061895,83.975780,94.13,97.04,91.33,Medium (1000-2000)
2,Figueroa High School,District,"$1,884,411.00",2949,$639.00,76.711767,81.158020,65.99,80.74,53.20,Large (2000-5000)
3,Ford High School,District,"$1,763,916.00",2739,$644.00,77.102592,80.746258,68.31,79.30,54.29,Large (2000-5000)
4,Griffin High School,Charter,"$917,500.00",1468,$625.00,83.351499,83.816757,93.39,97.14,90.60,Medium (1000-2000)
5,Hernandez High School,District,"$3,022,020.00",4635,$652.00,77.289752,80.934412,66.75,80.86,53.53,Large (2000-5000)
6,Holden High School,Charter,"$248,087.00",427,$581.00,83.803279,83.814988,92.51,96.25,89.23,Small (<1000)
7,Huang High School,District,"$1,910,635.00",2917,$655.00,76.629414,81.182722,65.68,81.32,53.51,Large (2000-5000)
8,Johnson High School,District,"$3,094,650.00",4761,$650.00,77.072464,80.966394,66.06,81.22,53.54,Large (2000-5000)
9,Pena High School,Charter,"$585,858.00",962,$609.00,83.839917,84.044699,94.59,95.95,90.54,Small (<1000)


In [532]:

# If the data type is not string, convert it to string
per_school_summary["Students Passing Math Percent"] = per_school_summary["Students Passing Math Percent"].astype(str).str.rstrip('%').astype(float)
per_school_summary["Students Passing Reading Percent"] = per_school_summary["Students Passing Reading Percent"].astype(str).str.rstrip('%').astype(float)
per_school_summary["Students Passing Both Percent"] = per_school_summary["Students Passing Both Percent"].astype(str).str.rstrip('%').astype(float)


# Calculate averages for the desired columns
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["Students Passing Math Percent"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["Students Passing Reading Percent"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["Students Passing Both Percent"].mean()



C:\Users\deebn\AppData\Local\Temp\ipykernel_17528\3685537809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_school_summary["Students Passing Math Percent"] = per_school_summary["Students Passing Math Percent"].astype(str).str.rstrip('%').astype(float)
C:\Users\deebn\AppData\Local\Temp\ipykernel_17528\3685537809.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_school_summary["Students Passing Reading Percent"] = per_school_summary["Students Passing Reading Percent"].astype(str).str.rstrip('%'

,Average Math Score,Average Reading Score,Average % Passing Math,Average % Passing Reading,Average % Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.55000,96.10000,89.885
Medium (1000-2000),83.374684,83.864438,93.59800,96.79000,90.620
Large (2000-5000),77.746417,81.344493,69.96375,82.76625,58.285


In [533]:
# Create a DataFrame to store the averages by school size
size_summary_df = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "Average % Passing Math": size_passing_math,
    "Average % Passing Reading": size_passing_reading,
    "Average % Overall Passing": size_overall_passing
})

# Display the DataFrame
size_summary_df

,Average Math Score,Average Reading Score,Average % Passing Math,Average % Passing Reading,Average % Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.55000,96.10000,89.885
Medium (1000-2000),83.374684,83.864438,93.59800,96.79000,90.620
Large (2000-5000),77.746417,81.344493,69.96375,82.76625,58.285


## Scores by School Type

In [536]:
# If the data type is not string, convert it to string
per_school_summary["Students Passing Math Percent"] = per_school_summary["Students Passing Math Percent"].astype(str).str.rstrip('%').astype(float)
per_school_summary["Students Passing Reading Percent"] = per_school_summary["Students Passing Reading Percent"].astype(str).str.rstrip('%').astype(float)
per_school_summary["Students Passing Both Percent"] = per_school_summary["Students Passing Both Percent"].astype(str).str.rstrip('%').astype(float)


# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["Students Passing Math Percent"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["Students Passing Reading Percent"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["Students Passing Both Percent"].mean()

C:\Users\deebn\AppData\Local\Temp\ipykernel_17528\1334901444.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_school_summary["Students Passing Math Percent"] = per_school_summary["Students Passing Math Percent"].astype(str).str.rstrip('%').astype(float)
C:\Users\deebn\AppData\Local\Temp\ipykernel_17528\1334901444.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_school_summary["Students Passing Reading Percent"] = per_school_summary["Students Passing Reading Percent"].astype(str).str.rstrip('%'

In [538]:
type_summary = pd.DataFrame({
    "Average Math Score": average_math_score_by_type,
    "Average Reading Score": average_reading_score_by_type,
    "Average % Passing Math": average_percent_passing_math_by_type,
    "Average % Passing Reading": average_percent_passing_reading_by_type,
    "Average % Overall Passing": average_percent_overall_passing_by_type
})

# Display results
type_summary

,Average Math Score,Average Reading Score,Average % Passing Math,Average % Passing Reading,Average % Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620000,96.586250,90.431250
District,76.956733,80.966636,66.548571,80.798571,53.671429
